# Network Science: Final Project

## Initialization

This includes:   
* Importing libraries
* Defining functions
* Loading data

### libraries

In [1]:
import networkx as nx #network analysis
import numpy as np 
import matplotlib.pyplot as plt #viz
import random 
import pandas as pd #to manipulate data frames
import math #for logarithm in entropy formula
from IPython.display import display
import os #files manipulation

#if you're working on google colab, uncomment the following line

# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir('/content/drive/My Drive/net-sci-project-master/net-sci-project-master/code')
# os.getwcd()

### functions

Functions to:  
* compute LCC
* compute I index
* compute R measure
* compute Interval E
* Apply Random Attack
* Apply Targeted Attack (takes a graph and sorted list of nodes and returns dictions of metrics: LCC, I, efficiency)
* Apply Community ased attack

In [2]:
def compute_lcc(G):
    if G.number_of_nodes() == 0:
        return 0
    else:
        max_comp= max(nx.connected_components(G), key=len)
        return len(max_comp)
    
def compute_i(G, i):
    if G.number_of_nodes() == 0:
        return 0
    else:
        max_comp= max(nx.connected_components(G), key=len)
        return len(max_comp)/i

def compute_r(g, sorted_nodes):
    G=g.copy()
    SUM=0
    N= G.number_of_nodes()

    if type(sorted_nodes[0])==tuple:

        for i in range(N):

            n=sorted_nodes[i][0]
            G.remove_node(n)
            lcc= compute_lcc(G)/N
            SUM+=lcc
    else:
        for i in range(N):

            n=sorted_nodes[i]
            G.remove_node(n)
            lcc= compute_lcc(G)/N
            SUM+=lcc

    return SUM/N

def compute_r_50_batches(attack):
    '''takes a tuple of dictionaries (output of attack) that contains the lcc of attack'''
    lcc_attack=attack[3]
    r = sum(list(lcc_attack.values()))/len(list(lcc_attack.values()))
    return r

def compute_r_100_batches(g, sorted_nodes):
    G=g.copy()
    N=G.number_of_nodes()
    batch_size = N//100
    SUM=0
    if type(sorted_nodes[0])==tuple:

        for i in range(50):
            for j in range(batch_size):
                n = sorted_nodes[i*batch_size+j][0]
                G.remove_node(n)
            lcc = compute_lcc(G)/N
            SUM+=lcc
    
    else:
        for i in range(50):
            for j in range(batch_size):
                n = sorted_nodes[i*batch_size+j]
                G.remove_node(n)
            lcc = compute_lcc(G)/N
            SUM+=lcc

    return SUM/100


def random_attack(g):
    G=g.copy()
    nodes = list(G.nodes())
    random.shuffle(nodes)
    #divide the nodes into 100 batches
    fix=G.number_of_nodes()
    batch_size = fix//100
    LCC={}
    Inter={}
    E={}
    lcc_i = max([len(c) for c in nx.connected_components(G)])

    for i in range(50):
        for j in range(batch_size):
            n = nodes[i*batch_size+j]
            G.remove_node(n)
        lcc = compute_lcc(G)
        i_index= compute_i(G, lcc_i)
        LCC[i] = lcc/fix
        Inter[i]=i_index
        E[i]=nx.algorithms.global_efficiency(G)
    return LCC, Inter, E

def random_attack_edge(g):
    '''returns the graph after a random attack of the edge'''
    G = g.copy()
    edges = list(G.edges())
    random.shuffle(edges)
    fix=G.number_of_nodes()
    batch_size = fix//100
    LCC={}
    Inter={}
    E={}
    lcc_i = max([len(c) for c in nx.connected_components(G)])

    for i in range(50):
        for j in range(batch_size):
            e = edges[i*batch_size+j]
            G.remove_edge(e[0], e[1])
        lcc = compute_lcc(G)
        i_index= compute_i(G, lcc_i)
        LCC[i] = lcc/fix
        Inter[i]=i_index
        E[i]=nx.algorithms.global_efficiency(G)
    return LCC, Inter, E

def attack(g, sorted_nodes):
    G=g.copy()
    fix=G.number_of_nodes()
    batch_size = fix//100
    norm_LCC={}
    LCC={}
    Inter={}
    E={}
    lcc_i = max([len(c) for c in nx.connected_components(G)])

    for i in range(50):
        for j in range(batch_size):
            n = sorted_nodes[i*batch_size+j]
            G.remove_node(n)
        lcc = compute_lcc(G)
        i_index= compute_i(G, lcc_i)
        norm_LCC[i] = lcc/fix
        LCC[i]=lcc
        Inter[i]=i_index
        E[i]=nx.algorithms.global_efficiency(G)
    return norm_LCC, Inter, E, LCC


### Loading networks

* Loading the network
* Extract some properties:
  * Compute statistics
  * Get degree distribution

In [3]:
arenas_email = nx.read_gml('../benchmark/arenas-email.gml', label='id')
bn_cat_mixed_species_brain_1 = nx.read_gml('../benchmark/bn-cat-mixed-species_brain_1.gml', label='id')
bn_macaque_rhesus_brain_2 = nx.read_gml('../benchmark/bn-macaque-rhesus_brain_2.gml', label='id')
circuits_s208 = nx.read_gml('../benchmark/circuits s208.gml', label='id')
circuits_s420 = nx.read_gml('../benchmark/circuits s420.gml', label='id')
circuits_s838 = nx.read_gml('../benchmark/circuits s838.gml', label='id')
dolphins = nx.read_gml('../benchmark/dolphins.gml', label='id')
e_coli = nx.read_gml('../benchmark/E. coli.gml', label='id')
facebook_0 = nx.read_gml('../benchmark/facebook 0.gml', label='id')
facebook_107 = nx.read_gml('../benchmark/facebook 107.gml', label='id')
facebook_1684 = nx.read_gml('../benchmark/facebook 1684.gml', label='id')
facebook_348 = nx.read_gml('../benchmark/facebook 348.gml', label='id')
facebook_414 = nx.read_gml('../benchmark/facebook 414.gml', label='id')
facebook_686 = nx.read_gml('../benchmark/facebook 686.gml', label='id')
fb_pages_food = nx.read_gml('../benchmark/fb-pages-food.gml', label='id')
karate = nx.read_gml('../benchmark/Karate.gml', label='id')
polbooks = nx.read_gml('../benchmark/polbooks.gml', label='id')
soc_firm_hi_tech = nx.read_gml('../benchmark/soc-firm-hi-tech.gml', label='id')
soc_tribes = nx.read_gml('../benchmark/soc-tribes.gml', label='id')
word_adjacencies = nx.read_gml('../benchmark/word_adjacencies.gml', label='id')

In [ ]:
net_911 = nx.read_edgelist('../benchmark/911.txt', nodetype=int)
corruption = nx.read_edgelist('../benchmark/corruption.txt', nodetype=int)
crime_net = nx.read_edgelist('../benchmark/CrimeNet.txt', nodetype=int)
digg = nx.read_edgelist('../benchmark/Digg.txt', nodetype=int)
email = nx.read_edgelist('../benchmark/Email.txt', nodetype=int)
jazz = nx.read_edgelist('../benchmark/Jazz.txt', nodetype=int)
petster_hamster = nx.read_edgelist('../benchmark/Petster-Petster.txt', nodetype=int)
router = nx.read_edgelist('../benchmark/Router.txt', nodetype=int)

#transfrm them to .gml files in benchmark folder
nx.write_gml(net_911, '../benchmark/911.gml')
nx.write_gml(corruption, '../benchmark/corruption.gml')
nx.write_gml(crime_net, '../benchmark/CrimeNet.gml')
nx.write_gml(digg, '../benchmark/Digg.gml')
nx.write_gml(email, '../benchmark/Email.gml')
nx.write_gml(jazz, '../benchmark/Jazz.gml')
nx.write_gml(petster_hamster, '../benchmark/Petster-Hamster.gml')
nx.write_gml(router, '../benchmark/Router.gml')

net_911=nx.read_gml('../benchmark/911.gml', label='id')
corruption=nx.read_gml('../benchmark/corruption.gml', label='id')
crime_net=nx.read_gml('../benchmark/CrimeNet.gml', label='id')
digg=nx.read_gml('../benchmark/Digg.gml', label='id')
email=nx.read_gml('../benchmark/Email.gml', label='id')
jazz=nx.read_gml('../benchmark/Jazz.gml', label='id')
petster_hamster=nx.read_gml('../benchmark/Petster-Hamster.gml', label='id')
router=nx.read_gml('../benchmark/Router.gml', label='id')


## Attacks

### Random Attack

Random attacks applied to all the networks, all are divided to 100 batches, attack is up to 50% of the nodes/edges:
* nodes attacks 
* edges attacks

In [ ]:
#--------------------------------------------------------------------------------------------------------------#
#nodes random attacks:

random_arenas_email_attack = random_attack(arenas_email)
random_bn_cat_mixed_species_brain_1_attack = random_attack(bn_cat_mixed_species_brain_1)
random_bn_macaque_rhesus_brain_2_attack = random_attack(bn_macaque_rhesus_brain_2)
random_circuits_s208_attack = random_attack(circuits_s208)
random_circuits_s420_attack = random_attack(circuits_s420)
random_circuits_s838_attack = random_attack(circuits_s838)
random_dolphins_attack = random_attack(dolphins)
random_e_coli_attack = random_attack(e_coli)
random_facebook_0_attack = random_attack(facebook_0)
random_facebook_107_attack = random_attack(facebook_107)
random_facebook_1684_attack = random_attack(facebook_1684)
random_facebook_348_attack = random_attack(facebook_348)
random_facebook_414_attack = random_attack(facebook_414)
random_facebook_686_attack = random_attack(facebook_686)
random_fb_pages_food_attack = random_attack(fb_pages_food)
random_karate_attack = random_attack(karate)
random_polbooks_attack = random_attack(polbooks)
random_soc_firm_hi_tech_attack = random_attack(soc_firm_hi_tech)
random_soc_tribes_attack = random_attack(soc_tribes)
random_word_adjacencies_attack = random_attack(word_adjacencies)

random_net911_attack = random_attack(net_911)
random_corruption_attack = random_attack(corruption)
random_crime_net_attack = random_attack(crime_net)
random_email_attack = random_attack(email)
random_jazz_attack = random_attack(jazz)
random_petster_hamster_attack = random_attack(petster_hamster)
random_router_attack = random_attack(router)
# random_digg_attack = random_attack(digg)

#--------------------------------------------------------------------------------------------------------------#
#edges random attacks:

random_arenas_email_attack_edge= random_attack_edge(arenas_email)
random_bn_cat_mixed_species_brain_1_attack_edge= random_attack_edge(bn_cat_mixed_species_brain_1)
random_bn_macaque_rhesus_brain_2_attack_edge= random_attack_edge(bn_macaque_rhesus_brain_2)
random_circuits_s208_attack_edge= random_attack_edge(circuits_s208)
random_circuits_s420_attack_edge= random_attack_edge(circuits_s420)
random_circuits_s838_attack_edge= random_attack_edge(circuits_s838)
random_dolphins_attack_edge= random_attack_edge(dolphins)
random_e_coli_attack_edge= random_attack_edge(e_coli)
random_facebook_0_attack_edge= random_attack_edge(facebook_0)
random_facebook_107_attack_edge= random_attack_edge(facebook_107)
random_facebook_1684_attack_edge= random_attack_edge(facebook_1684)
random_facebook_348_attack_edge= random_attack_edge(facebook_348)
random_facebook_414_attack_edge= random_attack_edge(facebook_414)
random_facebook_686_attack_edge= random_attack_edge(facebook_686)
random_fb_pages_food_attack_edge= random_attack_edge(fb_pages_food)
random_karate_attack_edge= random_attack_edge(karate)
random_polbooks_attack_edge= random_attack_edge(polbooks)
random_soc_firm_hi_tech_attack_edge= random_attack_edge(soc_firm_hi_tech)
random_soc_tribes_attack_edge= random_attack_edge(soc_tribes)
random_word_adjacencies_attack_edge= random_attack_edge(word_adjacencies)

random_net911_attack_edge= random_attack_edge(net_911)
random_corruption_attack_edge= random_attack_edge(corruption)
random_crime_net_attack_edge= random_attack_edge(crime_net)
random_email_attack_edge= random_attack_edge(email)
random_jazz_attack_edge= random_attack_edge(jazz)
random_petster_hamster_attack_edge= random_attack_edge(petster_hamster)
random_router_attack_edge= random_attack_edge(router)
# random_digg_attack_edge= random_attack_edge(digg)

#--------------------------------------------------------------------------------------------------------------#

### Centrality-based Targeted Attack
---------------------------

* Nodes centralities:
  * Betweenness centrality
  * Closeness centrality
  * Degree centrality
  * PLCi centrality
  * Percolaion centrality
  * Proximity centrality
  * Mapping Entropy Betweenness centrality
  * Mapping Entropy Closeness centrality
  * Mapping Entropy Degree centrality   

----------------

* Edges centralities:
  * Betweenness centrality